In [2]:
from ultralytics import YOLO

# --- 1. Cargar el modelo base yolo11n.pt ---
# Asumiendo que tienes este archivo y es compatible.
try:
    model = YOLO('yolo11n.pt') # Carga los pesos base de yolo11n
    print("Modelo base 'yolo11n.pt' cargado correctamente.")
except Exception as e:
    print(f"Error cargando el modelo base 'yolo11n.pt': {e}")
    print("Asegúrate de que el archivo existe y es compatible con Ultralytics.")
    exit()

# --- 2. Iniciar Entrenamiento ---
# Entrena el modelo usando tu archivo 'matriculas_solo.yaml'
# Este YAML DEBE tener nc: 1 y names: ['matricula']
results = model.train(
    data='matriculas.yaml',    # Ruta al archivo YAML (solo 1 clase: matricula)
    epochs=70,                    # Número de épocas (ajusta según necesites)
    imgsz=640,                    # Tamaño de imagen
    batch=4,                     # Tamaño del lote
    name='modelo_matriculas_yolo11', # Nombre para la carpeta de resultados
    exist_ok=True                 # Permite sobrescribir
)

print("\n¡Entrenamiento para SOLO matrículas (base yolo11n) completado!")
# La ruta real donde se guardan los pesos
print(f"Mejores pesos del modelo guardados en: runs/detect/{results.save_dir}/weights/best.pt")

Modelo base 'yolo11n.pt' cargado correctamente.
New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.215  Python-3.9.23 torch-2.8.0+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=matriculas.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=70, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mo

In [6]:
from ultralytics import YOLO
import cv2
import glob
import os
import matplotlib.pyplot as plt

# --- CONFIGURACIÓN DE MODELOS, dibujar imagen ---

# Carga el modelo general YOLOv8 pre-entrenado (para vehículos)
try:
    model_general_coches = YOLO('yolov8n.pt') # Usamos yolov8n estándar para coches
    print("Modelo general (yolov8n.pt) para coches cargado correctamente.")
except Exception as e:
    print(f"Error cargando modelo general 'yolov8n.pt': {e}")
    exit()

# Carga TU modelo entrenado (el que se basó en yolo11n.pt)
# ¡¡IMPORTANTE!! Ajusta esta ruta al 'best.pt' del entrenamiento anterior.
try:
    # Usa el nombre que le diste al entrenamiento ('modelo_matriculas_yolo11')
    ruta_modelo_matriculas_entrenado = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' # <--- AJUSTA ESTA RUTA
    model_matriculas = YOLO(ruta_modelo_matriculas_entrenado)
    print(f"Modelo entrenado para matrículas '{ruta_modelo_matriculas_entrenado}' cargado correctamente.")
except Exception as e:
    print(f"Error cargando tu modelo entrenado de matrículas '{ruta_modelo_matriculas_entrenado}': {e}")
    print("Asegúrate de que la ruta apunta al archivo 'best.pt' correcto.")
    exit()

# Clases de interés del modelo general (vehículos COCO)
coco_vehicle_classes = [2, 3, 5, 7] # car, motorcycle, bus, truck

# Colores y confianzas
color_vehiculo = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
conf_vehiculo = 0.40
conf_matricula = 0.1

# --- DEFINIR IMÁGENES DE PRUEBA Y SALIDA ---
ruta_carpeta_imagenes_test = './TGC_RBNW/imgenes_comprobar/*'
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
if not rutas_imagenes_test:
    print(f"Error: No se encontraron imágenes en '{ruta_carpeta_imagenes_test}'.")
    exit()
output_dir_deteccion_combinada = 'resultados2_deteccion_combinada_y11base'
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

print(f"Procesando {len(rutas_imagenes_test)} imágenes...")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

# --- PROCESAR IMÁGENES Y DIBUJAR ---
for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path)
    # print(f"Procesando: {base_filename}") # Puedes descomentar para ver el progreso

    frame = cv2.imread(img_path)
    if frame is None: continue
    frame_annotated = frame.copy()

    # --- 1. Detectar VEHÍCULOS ---
    results_coches = model_general_coches.predict(frame, classes=coco_vehicle_classes, conf=conf_vehiculo, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            conf_v = box_veh.conf.cpu().numpy()[0]
            class_id_v = int(box_veh.cls.cpu().numpy()[0])
            label_veh = model_general_coches.names[class_id_v]
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo, 2)
            cv2.putText(frame_annotated, f'{label_veh} {conf_v:.2f}', (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo, 2)

    # --- 2. Detectar MATRÍCULAS ---
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            label_mat = model_matriculas.names[0] # Siempre será 'matricula'
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            cv2.putText(frame_annotated, f'{label_mat} {conf_m:.2f}', (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)

    # --- GUARDAR ---
    output_path = os.path.join(output_dir_deteccion_combinada, f"comb_{base_filename}")
    cv2.imwrite(output_path, frame_annotated)

# --- FINALIZACIÓN ---
print("\nProceso de detección combinada completado.")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

Modelo general (yolov8n.pt) para coches cargado correctamente.
Modelo entrenado para matrículas 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' cargado correctamente.
Procesando 6 imágenes...
Resultados guardados en: resultados2_deteccion_combinada_y11base

Proceso de detección combinada completado.
Resultados guardados en: resultados2_deteccion_combinada_y11base


In [8]:
from ultralytics import YOLO
import cv2
import glob
import os
import matplotlib.pyplot as plt

# --- CONFIGURACIÓN DE MODELOS, dibujar imagen ---
try:
    model_general_coches = YOLO('yolov8n.pt') 
    print("Modelo general (yolov8n.pt) para coches cargado correctamente.")
except Exception as e:
    print(f"Error cargando modelo general 'yolov8n.pt': {e}")
    exit()

try:
    ruta_modelo_matriculas_entrenado = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt'
    model_matriculas = YOLO(ruta_modelo_matriculas_entrenado)
    print(f"Modelo entrenado para matrículas '{ruta_modelo_matriculas_entrenado}' cargado correctamente.")
except Exception as e:
    print(f"Error cargando tu modelo entrenado de matrículas '{ruta_modelo_matriculas_entrenado}': {e}")
    exit()

coco_vehicle_classes = [2, 3, 5, 7] # car, motorcycle, bus, truck
color_vehiculo = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
conf_vehiculo = 0.40
conf_matricula = 0.1

# --- DEFINIR VIDEO DE ENTRADA Y SALIDA ---
ruta_video_entrada = './input_media/video.mp4' 
ruta_video_salida = './resultados_video/video_detectado.mp4'
os.makedirs('./resultados_video', exist_ok=True) 

# --- CONFIGURAR LECTURA Y ESCRITURA DE VIDEO ---
cap = cv2.VideoCapture(ruta_video_entrada)
if not cap.isOpened():
    print(f"Error: No se pudo abrir el video '{ruta_video_entrada}'")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter(ruta_video_salida, fourcc, fps, (frame_width, frame_height))

print(f"Procesando video: {ruta_video_entrada}...")
print("--- ¡Pulsa 'q' en la ventana emergente para salir! ---")

# --- PROCESAR VIDEO FRAME A FRAME ---
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_annotated = frame.copy()

    # --- 1. Detectar VEHÍCULOS ---
    results_coches = model_general_coches.predict(frame, classes=coco_vehicle_classes, conf=conf_vehiculo, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            conf_v = box_veh.conf.cpu().numpy()[0]
            class_id_v = int(box_veh.cls.cpu().numpy()[0])
            label_veh = model_general_coches.names[class_id_v]
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo, 2)
            cv2.putText(frame_annotated, f'{label_veh} {conf_v:.2f}', (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo, 2)

    # --- 2. Detectar MATRÍCULAS ---
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            label_mat = model_matriculas.names[0] 
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            cv2.putText(frame_annotated, f'{label_mat} {conf_m:.2f}', (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)

    # --- GUARDAR EL FOTOGRAMA PROCESADO ---
    out.write(frame_annotated)
    
    # --- NUEVO: MOSTRAR EL VIDEO EN VENTANA EMERGENTE ---
    # Muestra el fotograma anotado en una ventana llamada 'Deteccion en Vivo'
    cv2.imshow('Deteccion en Vivo', frame_annotated)

    # --- NUEVO: ESPERAR 1ms Y COMPROBAR SI SE PULSA LA TECLA 'q' ---
    # Esto es OBLIGATORIO para que cv2.imshow funcione
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Detenido por el usuario.")
        break

# --- FINALIZACIÓN ---
# Liberar los objetos de video
cap.release()
out.release()
# --- NUEVO: CERRAR TODAS LAS VENTANAS DE OPENCV ---
cv2.destroyAllWindows()

print("\nProceso de video completado.")
print(f"Video guardado en: {ruta_video_salida}")

Modelo general (yolov8n.pt) para coches cargado correctamente.
Modelo entrenado para matrículas 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' cargado correctamente.
Procesando video: ./input_media/video.mp4...
--- ¡Pulsa 'q' en la ventana emergente para salir! ---

Proceso de video completado.
Video guardado en: ./resultados_video/video_detectado.mp4


**EASYOCR**

In [4]:
from ultralytics import YOLO
import cv2
import glob
import os
import matplotlib.pyplot as plt
import time
import csv
import easyocr

print("Cargando modelos YOLO...")
model_general_coches = YOLO('yolov8n.pt')
model_matriculas = YOLO('runs/detect/modelo_matriculas_yolo11/weights/best.pt')

print("Cargando EasyOCR...")
try:
    reader_easyocr = easyocr.Reader(['es', 'en'], gpu=False)
    print("EasyOCR cargado correctamente (CPU).")
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()

# --- RUTA CORREGIDA ---
ruta_carpeta_imagenes_test = './TGC_RBNW/imgenes_comprobar/*' # <--- ¡Corregido!
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
output_dir_deteccion_combinada = 'resultados_comparativa_OCR'
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

csv_output_path = os.path.join(output_dir_deteccion_combinada, 'easyocr_results.csv')
csv_results = []
csv_header = ['Imagen', 'Conf_Deteccion', 'Matricula_EasyOCR', 'Tiempo_EasyOCR_ms']

print(f"Procesando {len(rutas_imagenes_test)} imágenes para EasyOCR...")

for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path)
    frame = cv2.imread(img_path)
    if frame is None: continue
    frame_annotated = frame.copy() 

    # --- 1. Detectar VEHÍCULOS ---
    results_coches = model_general_coches.predict(frame, classes=[2, 3, 5, 7], conf=0.40, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), (255, 0, 0), 2)

    # --- 2. Detectar MATRÍCULAS ---
    results_matricula = model_matriculas.predict(frame, conf=0.1, verbose=False)
    
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            
            matricula_recortada = frame[max(0, my1 - 5):min(frame.shape[0], my2 + 5), max(0, mx1 - 5):min(frame.shape[1], mx2 + 5)]
            
            texto_easyocr = "N/A"
            tiempo_easyocr = 0.0

            try:
                t_start_ocr = time.time()
                ocr_result = reader_easyocr.readtext(matricula_recortada, detail=0, allowlist='0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ')
                t_end_ocr = time.time()
                tiempo_easyocr = (t_end_ocr - t_start_ocr) * 1000
                if ocr_result:
                    texto_easyocr = ocr_result[0].upper().replace(" ", "")
            except Exception as e:
                print(f"Error en EasyOCR: {e}")
                
            csv_results.append([base_filename, f"{conf_m:.2f}", texto_easyocr, f"{tiempo_easyocr:.1f}"])
            
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), (0, 255, 0), 2)
            label_final = f'{texto_easyocr} ({conf_m:.2f})'
            cv2.putText(frame_annotated, label_final, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            print(f"  > {base_filename} [Conf: {conf_m:.2f}] - EasyOCR: {texto_easyocr} ({tiempo_easyocr:.1f} ms)")

    output_path = os.path.join(output_dir_deteccion_combinada, f"easyocr_{base_filename}")
    cv2.imwrite(output_path, frame_annotated)

try:
    with open(csv_output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_header)
        writer.writerows(csv_results)
except Exception as e:
    print(f"Error al guardar el CSV: {e}")

print(f"\n--- Proceso EasyOCR completado ---")
print(f"Resultados CSV guardados en: {csv_output_path}")

Using CPU. Note: This module is much faster with a GPU.


Cargando modelos YOLO...
Cargando EasyOCR...
EasyOCR cargado correctamente (CPU).
Procesando 6 imágenes para EasyOCR...
  > img_comprobar1.jpg [Conf: 0.95] - EasyOCR: 6628GXR (172.8 ms)
  > img_comprobar1.jpg [Conf: 0.30] - EasyOCR: 15532BYP (124.5 ms)
  > img_comprobar2.jpg [Conf: 0.84] - EasyOCR: JPGC7085N (157.2 ms)
  > img_comprobar3.jpeg [Conf: 0.98] - EasyOCR: E (537.3 ms)
  > img_comprobar4.jpg [Conf: 0.91] - EasyOCR: JNV (213.7 ms)
  > img_comprobar4.jpg [Conf: 0.57] - EasyOCR: 3232JCJ (112.1 ms)
  > img_comprobar4.jpg [Conf: 0.18] - EasyOCR: N/A (70.4 ms)
  > img_comprobar5.jpeg [Conf: 0.93] - EasyOCR: JHM (427.8 ms)
  > img_comprobar6.jpeg [Conf: 0.94] - EasyOCR: GGAGX7497 (175.2 ms)
  > img_comprobar6.jpeg [Conf: 0.21] - EasyOCR: N/A (33.6 ms)

--- Proceso EasyOCR completado ---
Resultados CSV guardados en: resultados_comparativa_OCR\easyocr_results.csv


**TESSERACT**

In [5]:
from ultralytics import YOLO
import cv2
import glob
import os
import matplotlib.pyplot as plt
import time
import csv
import pytesseract # <-- NUEVO: Importar Tesseract

# --- ¡¡IMPORTANTE!! CONFIGURACIÓN DE TESSERACT ---
# Tienes que decirle a Python dónde está el archivo .exe que instalaste.
# ¡¡AJUSTA ESTA RUTA A LA DE TU ORDENADOR!!
try:
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\lucia\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
    version = pytesseract.get_tesseract_version()
    print(f"Tesseract {version} cargado correctamente.")
except Exception as e:
    print(f"Error cargando Tesseract. ¿Instalaste el programa y configuraste bien la ruta?")
    print(f"Error: {e}")
    # exit() # Descomenta esto si quieres que el script pare si Tesseract falla

# --- CONFIGURACIÓN DE MODELOS ---
print("Cargando modelos YOLO...")
model_general_coches = YOLO('yolov8n.pt')
model_matriculas = YOLO('runs/detect/modelo_matriculas_yolo11/weights/best.pt')


# --- CONFIGURACIÓN DE RUTAS ---
ruta_carpeta_imagenes_test = './TGC_RBNW/imgenes_comprobar/*' # Ruta corregida
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
output_dir_deteccion_combinada = 'resultados_comparativa_OCR'
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

# --- CONFIGURACIÓN CSV ---
csv_output_path = os.path.join(output_dir_deteccion_combinada, 'tesseract_results.csv')
csv_results = []
csv_header = ['Imagen', 'Conf_Deteccion', 'Matricula_Tesseract', 'Tiempo_Tesseract_ms']

print(f"Procesando {len(rutas_imagenes_test)} imágenes para Tesseract...")

# --- Configuración de Tesseract para matrículas ---
# --psm 7: Trata la imagen como una sola línea de texto.
# Whitelist: Solo busca estos caracteres.
config_tesseract = '--psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ'

# --- PROCESAR IMÁGENES ---
for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path)
    frame = cv2.imread(img_path)
    if frame is None: continue
    frame_annotated = frame.copy() 

    # --- 1. Detectar VEHÍCULOS (Opcional) ---
    results_coches = model_general_coches.predict(frame, classes=[2, 3, 5, 7], conf=0.40, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), (255, 0, 0), 2)

    # --- 2. Detectar MATRÍCULAS ---
    results_matricula = model_matriculas.predict(frame, conf=0.1, verbose=False)
    
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            
            matricula_recortada = frame[max(0, my1 - 5):min(frame.shape[0], my2 + 5), max(0, mx1 - 5):min(frame.shape[1], mx2 + 5)]
            
            texto_tesseract = "N/A"
            tiempo_tesseract = 0.0

            # --- Lógica de Tesseract ---
            try:
                t_start_ocr = time.time()
                # Pasamos la imagen recortada (en formato OpenCV/BGR)
                # y la configuración que definimos antes.
                texto_tesseract = pytesseract.image_to_string(matricula_recortada, config=config_tesseract)
                t_end_ocr = time.time()
                tiempo_tesseract = (t_end_ocr - t_start_ocr) * 1000 # en ms
                
                # Limpiamos el resultado (Tesseract suele añadir saltos de línea \n o \f)
                texto_tesseract = texto_tesseract.strip().upper().replace(" ", "")
                
                if not texto_tesseract:
                    texto_tesseract = "N/A" # Si está vacío, lo marcamos N/A
                    
            except Exception as e:
                print(f"Error en Tesseract: {e}")
                
            csv_results.append([base_filename, f"{conf_m:.2f}", texto_tesseract, f"{tiempo_tesseract:.1f}"])
            
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), (255, 0, 255), 2) # Color Magenta para Tesseract
            label_final = f'T: {texto_tesseract} ({conf_m:.2f})'
            cv2.putText(frame_annotated, label_final, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
            print(f"  > {base_filename} [Conf: {conf_m:.2f}] - Tesseract: {texto_tesseract} ({tiempo_tesseract:.1f} ms)")

    output_path = os.path.join(output_dir_deteccion_combinada, f"tesseract_{base_filename}")
    cv2.imwrite(output_path, frame_annotated)

# --- GUARDAR EL ARCHIVO CSV ---
try:
    with open(csv_output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_header)
        writer.writerows(csv_results)
except Exception as e:
    print(f"Error al guardar el CSV: {e}")

print(f"\n--- Proceso Tesseract completado ---")
print(f"Resultados CSV guardados en: {csv_output_path}")

Tesseract 5.4.0.20240606 cargado correctamente.
Cargando modelos YOLO...
Procesando 6 imágenes para Tesseract...
  > img_comprobar1.jpg [Conf: 0.95] - Tesseract: N/A (111.5 ms)
  > img_comprobar1.jpg [Conf: 0.30] - Tesseract: A5532BYP (94.4 ms)
  > img_comprobar2.jpg [Conf: 0.84] - Tesseract: IPGC7089N (113.0 ms)
  > img_comprobar3.jpeg [Conf: 0.98] - Tesseract: 13168JHM (162.5 ms)
  > img_comprobar4.jpg [Conf: 0.91] - Tesseract: N/A (116.6 ms)
  > img_comprobar4.jpg [Conf: 0.57] - Tesseract: 3232NC9 (83.5 ms)
  > img_comprobar4.jpg [Conf: 0.18] - Tesseract: KA (116.3 ms)
  > img_comprobar5.jpeg [Conf: 0.93] - Tesseract: N/A (136.2 ms)
  > img_comprobar6.jpeg [Conf: 0.94] - Tesseract: N/A (100.0 ms)
  > img_comprobar6.jpeg [Conf: 0.21] - Tesseract: N/A (117.1 ms)

--- Proceso Tesseract completado ---
Resultados CSV guardados en: resultados_comparativa_OCR\tesseract_results.csv


In [1]:
import cv2
import os
import csv
from ultralytics import YOLO
import easyocr

# --- 1. CONFIGURACIÓN INICIAL ---

# Cargar Modelos YOLO
print("Cargando modelos YOLO...")
try:
    model_general_coches = YOLO('yolov8n.pt')
    ruta_modelo_matriculas = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt'
    model_matriculas = YOLO(ruta_modelo_matriculas)
    print("Modelos YOLO cargados.")
except Exception as e:
    print(f"Error cargando modelos YOLO: {e}")
    exit()

# Cargar EasyOCR (solo se hace una vez)
print("Cargando EasyOCR (esto puede tardar un momento)...")
try:
    reader_easyocr = easyocr.Reader(['es', 'en'], gpu=False) # False si no tienes GPU
    print("EasyOCR cargado.")
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()

# Clases de interés (P4): 0=person, 2=car, 3=motorcycle, 5=bus, 7=truck
coco_classes_of_interest = [0, 2, 3, 5, 7] 

# Colores y confianzas
color_vehiculo_persona = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
conf_vehiculo = 0.40
conf_matricula = 0.1

# --- 2. CONFIGURACIÓN DE ENTRADA Y SALIDA ---

# Rutas de Vídeo
ruta_video_entrada = './input_media/video.mp4'
ruta_video_salida = './resultados_video/video_final_con_ocr.mp4'
os.makedirs('./resultados_video', exist_ok=True)

# Rutas de CSV (P4.6 y P4b)
csv_output_path = './resultados_video/log_detecciones.csv'
# Definimos el header del CSV
csv_header = [
    'fotograma', 
    'tipo_objeto',      # person, car, matricula
    'confianza', 
    'id_tracking',      # 'N/A' para matrículas
    'x1', 'y1', 'x2', 'y2', 
    'texto_matricula'   # 'N/A' para personas/vehículos
]

# --- 3. INICIALIZAR LECTORES Y ESCRITORES ---

# Lector de Video
cap = cv2.VideoCapture(ruta_video_entrada)
if not cap.isOpened():
    print(f"Error: No se pudo abrir el video '{ruta_video_entrada}'")
    exit()

# Escritor de Video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(ruta_video_salida, fourcc, fps, (frame_width, frame_height))

# Escritor de CSV
try:
    csv_file = open(csv_output_path, 'w', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_header)
except Exception as e:
    print(f"Error al abrir el CSV: {e}")
    exit()

# Variables de conteo y tracking (P4)
frame_num = 0
# Usamos 'sets' para contar solo los IDs únicos que aparecen
tracked_ids = {
    'person': set(),
    'car': set(),
    'motorcycle': set(),
    'bus': set(),
    'truck': set()
}

print(f"Procesando video: {ruta_video_entrada}...")
print("--- ¡Pulsa 'q' en la ventana emergente para salir! ---")

# --- 4. PROCESAR VIDEO FRAME A FRAME ---

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_num += 1
    frame_annotated = frame.copy()

    # --- Tarea P4: Detectar y SEGUIR (Track) Vehículos y Personas ---
    # Usamos model.track() en lugar de model.predict()
    results_general = model_general_coches.track(
        frame, 
        classes=coco_classes_of_interest, 
        conf=conf_vehiculo, 
        verbose=False, 
        persist=True # ¡Importante para mantener el tracking entre frames!
    )

    if results_general[0].boxes.id is not None:
        # Iteramos sobre las cajas (boxes) del resultado
        for box in results_general[0].boxes:
            # Coordenadas y datos
            coords = box.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords
            conf_v = box.conf.cpu().numpy()[0]
            class_id_v = int(box.cls.cpu().numpy()[0])
            track_id = int(box.id.cpu().numpy()[0]) # <-- ¡Aquí está el ID de Tracking!
            label_veh = model_general_coches.names[class_id_v]

            # Tarea P4 (Conteo): Añadimos el ID al 'set'
            if label_veh in tracked_ids:
                tracked_ids[label_veh].add(track_id)

            # Dibujar en el frame
            label_display = f'ID: {track_id} ({label_veh})'
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo_persona, 2)
            cv2.putText(frame_annotated, label_display, (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo_persona, 2)

            # Tarea P4 (CSV): Escribir en el CSV
            # 'N/A' porque este objeto no es una matrícula
            csv_writer.writerow([frame_num, label_veh, f"{conf_v:.2f}", track_id, x1_v, y1_v, x2_v, y2_v, 'N/A'])


    # --- Tarea P4b: Detectar Matrículas y hacer OCR ---
    # Para esto usamos predict(), no necesitamos tracking de la matrícula
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            label_mat = model_matriculas.names[0] # 'matricula'

            # Tarea P4b (OCR): Recortar la matrícula y leerla
            # Añadimos un pequeño margen (padding) como en tu script de OCR
            crop_matricula = frame[
                max(0, my1 - 5) : min(frame.shape[0], my2 + 5),
                max(0, mx1 - 5) : min(frame.shape[1], mx2 + 5)
            ]
            
            texto_ocr = "N/A"
            if crop_matricula.size > 0: # Asegurarse de que el recorte no esté vacío
                try:
                    # Usamos la whitelist de tu script de OCR
                    ocr_result = reader_easyocr.readtext(crop_matricula, detail=0, allowlist='0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ')
                    if ocr_result:
                        texto_ocr = ocr_result[0].upper().replace(" ", "")
                except Exception as e:
                    print(f"Error en EasyOCR en frame {frame_num}: {e}")
            
            # Dibujar en el frame (P4b)
            label_display_ocr = f'{texto_ocr} ({conf_m:.2f})'
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            cv2.putText(frame_annotated, label_display_ocr, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)

            # Tarea P4b (CSV): Escribir en el CSV
            # 'N/A' porque no tenemos un ID de tracking para la matrícula
            csv_writer.writerow([frame_num, label_mat, f"{conf_m:.2f}", 'N/A', mx1, my1, mx2, my2, texto_ocr])


    # --- 5. GUARDAR Y MOSTRAR ---
    
    # Guardar el fotograma procesado
    out.write(frame_annotated)
    
    # Mostrar el video en ventana emergente
    cv2.imshow('Deteccion en Vivo (P4 + P4b)', frame_annotated)

    # Esperar 1ms y comprobar si se pulsa la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Detenido por el usuario.")
        break

# --- 6. FINALIZACIÓN ---
print("\nLimpiando recursos...")
# Liberar los objetos de video
cap.release()
out.release()
# Cerrar el archivo CSV
csv_file.close()
# Cerrar todas las ventanas de OpenCV
cv2.destroyAllWindows()

# Imprimir el conteo final (P4)
print("\n--- Conteo Final (IDs únicos detectados) ---")
total_general = 0
for label, id_set in tracked_ids.items():
    count = len(id_set)
    print(f"Total {label}s: {count}")
    total_general += count
print(f"Total objetos seguidos: {total_general}")

print(f"\nProceso de video completado.")
print(f"Video guardado en: {ruta_video_salida}")
print(f"Log CSV guardado en: {csv_output_path}")

Cargando modelos YOLO...


Using CPU. Note: This module is much faster with a GPU.


Modelos YOLO cargados.
Cargando EasyOCR (esto puede tardar un momento)...
EasyOCR cargado.
Procesando video: ./input_media/video.mp4...
--- ¡Pulsa 'q' en la ventana emergente para salir! ---
Detenido por el usuario.

Limpiando recursos...

--- Conteo Final (IDs únicos detectados) ---
Total persons: 9
Total cars: 24
Total motorcycles: 2
Total buss: 4
Total trucks: 6
Total objetos seguidos: 45

Proceso de video completado.
Video guardado en: ./resultados_video/video_final_con_ocr.mp4
Log CSV guardado en: ./resultados_video/log_detecciones.csv


In [ ]:
import cv2
import os
import csv
from ultralytics import YOLO
import easyocr

# --- 1. CONFIGURACIÓN INICIAL ---

# Cargar Modelos YOLO
print("Cargando modelos YOLO...")
try:
    model_general_coches = YOLO('yolov8n.pt')
    ruta_modelo_matriculas = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt'
    model_matriculas = YOLO(ruta_modelo_matriculas)
    print("Modelos YOLO cargados.")
except Exception as e:
    print(f"Error cargando modelos YOLO: {e}")
    exit()

# Cargar EasyOCR (solo se hace una vez)
print("Cargando EasyOCR (esto puede tardar un momento)...")
try:
    reader_easyocr = easyocr.Reader(['es', 'en'], gpu=False) # False si no tienes GPU
    print("EasyOCR cargado.")
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()

# Clases de interés (P4): 0=person, 2=car, 3=motorcycle, 5=bus, 7=truck
coco_classes_of_interest = [0, 2, 3, 5, 7] 
# NUEVO: Clases que queremos contar al cruzar la línea
vehicle_classes_to_count = [2, 3, 5, 7] # car, motorcycle, bus, truck

# Colores y confianzas
color_vehiculo_persona = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
color_linea_conteo = (0, 0, 255) # Rojo
conf_vehiculo = 0.40
conf_matricula = 0.1

# --- 2. CONFIGURACIÓN DE ENTRADA Y SALIDA ---

# Rutas de Vídeo
ruta_video_entrada = './input_media/video.mp4'
ruta_video_salida = './resultados_video/video_final_con_ocr_conteo.mp4'
os.makedirs('./resultados_video', exist_ok=True)

# Rutas de CSV (P4.6 y P4b)
csv_output_path = './resultados_video/log_detecciones.csv'
# Definimos el header del CSV
csv_header = [
    'fotograma', 
    'tipo_objeto',      # person, car, matricula
    'confianza', 
    'id_tracking',      # 'N/A' para matrículas
    'x1', 'y1', 'x2', 'y2', 
    'texto_matricula'   # 'N/A' para personas/vehículos
]

# --- 3. INICIALIZAR LECTORES Y ESCRITORES ---

# Lector de Video
cap = cv2.VideoCapture(ruta_video_entrada)
if not cap.isOpened():
    print(f"Error: No se pudo abrir el video '{ruta_video_entrada}'")
    exit()

# Escritor de Video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(ruta_video_salida, fourcc, fps, (frame_width, frame_height))

# Escritor de CSV
try:
    csv_file = open(csv_output_path, 'w', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_header)
except Exception as e:
    print(f"Error al abrir el CSV: {e}")
    exit()

# Variables de conteo y tracking
frame_num = 0
# Usamos 'sets' para contar los IDs únicos que aparecen (CONTEO ANTIGUO, informativo)
tracked_ids = {
    'person': set(),
    'car': set(),
    'motorcycle': set(),
    'bus': set(),
    'truck': set()
}

# --- NUEVO: Variables para el Conteo por Cruce de Línea ---
# Definimos la línea de conteo (horizontal, al 75% de la altura)
line_y = int(frame_height * 0.75) 
# Diccionario para guardar la última posición 'y' de cada ID
# Formato: { track_id: ultima_y }
track_history = {} 
# Set para guardar los IDs que ya hemos contado
counted_ids = set() 
# El nuevo contador
vehicle_count = 0
# --- Fin de variables nuevas ---


print(f"Procesando video: {ruta_video_entrada}...")
print("--- ¡Pulsa 'q' en la ventana emergente para salir! ---")

# --- 4. PROCESAR VIDEO FRAME A FRAME ---

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_num += 1
    frame_annotated = frame.copy()
    
    # NUEVO: Set para guardar los IDs vistos en ESTE frame
    current_frame_ids = set()

    # --- Tarea P4: Detectar y SEGUIR (Track) Vehículos y Personas ---
    results_general = model_general_coches.track(
        frame, 
        classes=coco_classes_of_interest, 
        conf=conf_vehiculo, 
        verbose=False, 
        persist=True 
    )

    if results_general[0].boxes.id is not None:
        # Iteramos sobre las cajas (boxes) del resultado
        for box in results_general[0].boxes:
            # Coordenadas y datos
            coords = box.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords
            conf_v = box.conf.cpu().numpy()[0]
            class_id_v = int(box.cls.cpu().numpy()[0])
            track_id = int(box.id.cpu().numpy()[0]) # <-- ¡Aquí está el ID de Tracking!
            label_veh = model_general_coches.names[class_id_v]

            # NUEVO: Añadir ID al set de IDs del frame actual
            current_frame_ids.add(track_id)

            # --- NUEVO: Lógica de Conteo por Cruce de Línea ---
            # Usamos el punto inferior central del bounding box (y2_v)
            current_y = y2_v 
            
            # Solo contamos las clases de vehículos
            if class_id_v in vehicle_classes_to_count:
                # 1. Comprobar si ya estábamos siguiendo este ID
                if track_id in track_history:
                    # 2. Obtener su posición anterior
                    prev_y = track_history[track_id]
                    
                    # 3. Comprobar si ha cruzado la línea (de arriba hacia abajo)
                    #    Y si NO lo hemos contado ya
                    if prev_y < line_y and current_y >= line_y and track_id not in counted_ids:
                        vehicle_count += 1
                        counted_ids.add(track_id)
                        print(f"[CONTEO] Nuevo vehiculo (ID: {track_id}) cruzó. Total: {vehicle_count}")
                
                # 4. Actualizar el historial de posición para este ID
                track_history[track_id] = current_y
            # --- Fin de la lógica de conteo ---


            # Tarea P4 (Conteo ANTIGUO): Añadimos el ID al 'set'
            if label_veh in tracked_ids:
                tracked_ids[label_veh].add(track_id)

            # Dibujar en el frame
            label_display = f'ID: {track_id} ({label_veh})'
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo_persona, 2)
            cv2.putText(frame_annotated, label_display, (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo_persona, 2)

            # Tarea P4 (CSV): Escribir en el CSV
            csv_writer.writerow([frame_num, label_veh, f"{conf_v:.2f}", track_id, x1_v, y1_v, x2_v, y2_v, 'N/A'])

    
    # --- NUEVO: Limpiar el historial de IDs que ya no están en pantalla ---
    # Esto evita que el diccionario track_history crezca indefinidamente
    stale_ids = set(track_history.keys()) - current_frame_ids
    for stale_id in stale_ids:
        del track_history[stale_id]
    # --- Fin de la limpieza ---


    # --- Tarea P4b: Detectar Matrículas y hacer OCR ---
    # (Esta parte no cambia)
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            label_mat = model_matriculas.names[0] # 'matricula'

            # Tarea P4b (OCR)
            crop_matricula = frame[
                max(0, my1 - 5) : min(frame.shape[0], my2 + 5),
                max(0, mx1 - 5) : min(frame.shape[1], mx2 + 5)
            ]
            
            texto_ocr = "N/A"
            if crop_matricula.size > 0: 
                try:
                    ocr_result = reader_easyocr.readtext(crop_matricula, detail=0, allowlist='0123456789ABCDEFGHIJKLMNPQRSTUVWXYZ')
                    if ocr_result:
                        texto_ocr = ocr_result[0].upper().replace(" ", "")
                except Exception as e:
                    print(f"Error en EasyOCR en frame {frame_num}: {e}")
            
            # Dibujar en el frame (P4b)
            label_display_ocr = f'{texto_ocr} ({conf_m:.2f})'
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            cv2.putText(frame_annotated, label_display_ocr, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)

            # Tarea P4b (CSV)
            csv_writer.writerow([frame_num, label_mat, f"{conf_m:.2f}", 'N/A', mx1, my1, mx2, my2, texto_ocr])


    # --- 5. GUARDAR Y MOSTRAR ---
    
    # --- NUEVO: Dibujar la línea de conteo y el contador ---
    cv2.line(frame_annotated, (0, line_y), (frame_width, line_y), color_linea_conteo, 2)
    count_text = f"Vehiculos Contados: {vehicle_count}"
    # Poner el texto en un fondo sólido para mejor legibilidad
    (text_width, text_height), baseline = cv2.getTextSize(count_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)
    cv2.rectangle(frame_annotated, (10, 30), (10 + text_width, 30 + text_height + baseline), (0,0,0), -1) # Fondo negro
    cv2.putText(frame_annotated, count_text, (10, 30 + text_height), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color_linea_conteo, 3)
    # --- Fin de dibujar línea y contador ---

    # Guardar el fotograma procesado
    out.write(frame_annotated)
    
    # Mostrar el video en ventana emergente
    cv2.imshow('Deteccion y Conteo por Cruce de Linea', frame_annotated)

    # Esperar 1ms y comprobar si se pulsa la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Detenido por el usuario.")
        break

# --- 6. FINALIZACIÓN ---
print("\nLimpiando recursos...")
# Liberar los objetos de video
cap.release()
out.release()
# Cerrar el archivo CSV
csv_file.close()
# Cerrar todas las ventanas de OpenCV
cv2.destroyAllWindows()

# Imprimir el conteo final (NUEVO)
print("\n--- Conteo Final (Cruce de Línea) ---")
print(f"Total Vehiculos contados al cruzar la línea: {vehicle_count}")

# Imprimir el conteo antiguo (informativo)
print("\n--- Conteo (Total IDs Únicos detectados) ---")
total_general = 0
for label, id_set in tracked_ids.items():
    count = len(id_set)
    print(f"Total IDs unicos para {label}s: {count}")
    total_general += count
print(f"Total IDs unicos asignados: {total_general}")

print(f"\nProceso de video completado.")
print(f"Video guardado en: {ruta_video_salida}")
print(f"Log CSV guardado en: {csv_output_path}")

Using CPU. Note: This module is much faster with a GPU.


Cargando modelos YOLO...
Modelos YOLO cargados.
Cargando EasyOCR (esto puede tardar un momento)...
EasyOCR cargado.
Procesando video: ./input_media/video.mp4...
--- ¡Pulsa 'q' en la ventana emergente para salir! ---
Detenido por el usuario.

Limpiando recursos...

--- Conteo Final (Cruce de Línea) ---
Total Vehiculos contados al cruzar la línea: 0

--- Conteo (Total IDs Únicos detectados) ---
Total IDs unicos para persons: 1
Total IDs unicos para cars: 3
Total IDs unicos para motorcycles: 0
Total IDs unicos para buss: 1
Total IDs unicos para trucks: 0
Total IDs unicos asignados: 5

Proceso de video completado.
Video guardado en: ./resultados_video/video_final_con_ocr_conteo.mp4
Log CSV guardado en: ./resultados_video/log_detecciones.csv
